In [26]:
import os
import pandas as pd

In [ ]:
train_path = '/Users/ajones/Documents/DS340/ASL/ASL_Dataset/Train/'
os.chdir(train_path)
lists = os.listdir(train_path)
labels = []
file_lst = []

for folder in lists:
    files = os.listdir(train_path +"/"+folder)
    for file in files:
      path_file = train_path + "/" + folder + "/" + file
      file_lst.append(path_file)
      labels.append(folder)
      img_file = Image.open(path_file)
      width, height = img_file.size
      format = img_file.format
      mode = img_file.mode
      img_grey  = img_file.convert('L')
      value = np.asarray(img_grey.getdata(), dtype=np.float32).reshape((img_grey.size[1], img_grey.size[0]))
      value = value.flatten()
        
      with open("train.csv", 'a') as f:
          writer = csv.writer(f)
          writer.writerow(value)

In [ ]:
test_path = '/Users/ajones/Documents/DS340/ASL/ASL_Dataset/Test'
os.chdir(test_path)
lists = os.listdir(test_path)
labels = []
file_lst = []

for folder in lists:
    files = os.listdir(test_path +"/"+folder)
    for file in files:
      path_file = test_path + "/" + folder + "/" + file
      file_lst.append(path_file)
      labels.append(folder)
      img_file = Image.open(path_file)
      width, height = img_file.size
      format = img_file.format
      mode = img_file.mode
      img_grey  = img_file.convert('L')
      value = np.asarray(img_grey.getdata(), dtype=np.float32).reshape((img_grey.size[1], img_grey.size[0]))
      value = value.flatten()
        
      with open("train.csv", 'a') as f:
          writer = csv.writer(f)
          writer.writerow(value)